In [1]:
import numpy as np
# import cv2

from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from datasets import UnlabeledCOVIDxCT

%matplotlib inline

In [14]:
import pytorch_lightning as pl
from pl_bolts.models.self_supervised import Moco_v2
from pl_bolts.models.self_supervised.moco.transforms import \
    Moco2TrainImagenetTransforms, Moco2EvalImagenetTransforms
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from datasets import SSLCOVIDxCT

In [3]:
root = Path("images") / "samples"
imgs = list(root.glob("*.png"))

img_filepath = imgs[0]
img = Image.open(img_filepath)

In [4]:
# dataset = UnlabeledCOVIDxCT("/media/storage/COVIDxCT")
datamodule = SSLCOVIDxCT(data_dir=Path("/media") / "storage" / "COVIDxCT",
                         num_workers=2)
datamodule.train_transforms = Moco2TrainImagenetTransforms(height=224)
datamodule.val_transforms = Moco2EvalImagenetTransforms(height=224)
datamodule.setup()

train_dataloader = datamodule.train_dataloader()
val_dataloader = datamodule.val_dataloader()

In [5]:
for batch in train_dataloader:
    (img_1, img_2), label = batch
    # plt.imshow(np.asarray(x)
    break

In [3]:
df = UnlabeledCOVIDxCT("/media/storage/COVIDxCT").df
df.head()

,filename,class,xmin,ymin,xmax,ymax
0,CP_1216_3434_0256.png,1,57,127,483,478
1,CP_1459_3997_0030.png,1,30,105,488,414
2,Normal_1735_1030_0030.png,0,21,87,499,385
3,CP_595_2957_0256.png,1,0,0,512,394
4,CP_2452_2931_0224.png,1,0,80,512,424


In [15]:
labels = df.loc[:, "class"]
counts = labels.value_counts()
weights = 1. / counts
weights = weights[labels].to_numpy()
print(type(weights))
print()
print(weights[df["class"]])

<class 'numpy.ndarray'>

[3.92218387e-05 3.92218387e-05 3.92218387e-05 ... 3.92218387e-05
 3.92218387e-05 2.77808645e-05]


In [25]:
sampler = WeightedRandomSampler(weights, num_samples=len(df))
list(sampler)[:10]

[48491, 111483, 64102, 107060, 136551, 129786, 45829, 122363, 97265, 80863]